In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import importlib
import pandas as pd
import matplotlib.gridspec as gridspec
import time

import sys
sys.path.append('../scripts')

import sirs
import params
import plotting
from params import cols

from pylab import cm
import matplotlib as mpl
from matplotlib import colors

plotting.set_rcParams()

Code will be added shortly 